In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import io
from tensorflow.keras.models import load_model

# Path to the trained model
MODEL_PATH = "/content/drive/MyDrive/best_vgg16_model.h5"

# Load the model without recompiling
model = load_model(MODEL_PATH, compile=False)

# Print and save the model summary
buf = io.StringIO()
model.summary(print_fn=lambda x: buf.write(x + "\n"))
summary_str = buf.getvalue()

print(summary_str)  # print the entire summary
with open("/content/model_summary.txt", "w") as f:
    f.write(summary_str)


Model: "sequential_3"
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)              │ (None, 7, 7, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_3      │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 128)            │        65,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 128)       

In [3]:
#1. Import packages
import os, glob
import numpy as np
import cv2
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.layers import Input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input

#2. Paths and constants
IMG_DIR    = "/content/drive/MyDrive/feature"             # directory of images to visualize
OUT_DIR    = "/content/drive/MyDrive/feature_cam_vgg16"   # output directory
MODEL_PATH = "/content/drive/MyDrive/best_vgg16_model.h5" # trained VGG16 model
IMG_SIZE   = (224, 224)
os.makedirs(OUT_DIR, exist_ok=True)

#3. Load trained model
base_model = load_model(MODEL_PATH, compile=False)

#4. Extract VGG16 backbone and last convolutional layer
backbone = base_model.get_layer("vgg16")
LAST_CONV_NAME = "block5_conv3"
last_conv_layer = backbone.get_layer(LAST_CONV_NAME)

# Feature sub-model: input -> (last conv features, backbone output)
feat_model = Model(inputs=backbone.input,
                   outputs=[last_conv_layer.output, backbone.output],
                   name="vgg16_feat_model")

#5. Rebuild full forward graph: backbone output -> custom head
inp = Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3), name="gradcam_input")
conv_feat_tensor, bb_out = feat_model(inp)

# Attach custom classification head
x = bb_out
attach = False
for lyr in base_model.layers:
    if attach:
        x = lyr(x)
    if lyr.name == "vgg16":
        attach = True

func_model = Model(inputs=inp, outputs=x, name="vgg16_functional_rebuilt")
grad_model = Model(inputs=func_model.input,
                   outputs=[conv_feat_tensor, func_model.output])

#6. Grad-CAM core function
def make_gradcam_heatmap(img_array, use_logit=False):
    with tf.GradientTape() as tape:
        conv_out, preds = grad_model(img_array, training=False)
        p = preds[:, 0]  # Dense(1, sigmoid) — probability of AI
        if use_logit:
            eps = 1e-7
            p = tf.clip_by_value(p, eps, 1 - eps)
            target = tf.math.log(p / (1 - p))
        else:
            target = p
    grads = tape.gradient(target, conv_out)
    pooled = tf.reduce_mean(grads, axis=(0, 1, 2))
    conv_map = conv_out[0]
    heatmap = tf.reduce_sum(conv_map * pooled, axis=-1)
    heatmap = tf.nn.relu(heatmap)
    heatmap = heatmap / (tf.reduce_max(heatmap) + 1e-8)
    return heatmap.numpy(), float(p.numpy()[0])

def overlay_heatmap(heatmap, bgr_img, alpha=0.45):
    h, w = bgr_img.shape[:2]
    heatmap = cv2.resize(heatmap, (w, h))
    heatmap_uint8 = np.uint8(255 * heatmap)
    heatmap_color = cv2.applyColorMap(heatmap_uint8, cv2.COLORMAP_JET)
    return cv2.addWeighted(heatmap_color, alpha, bgr_img, 1 - alpha, 0)

#7. Batch processing, saving, and displaying results
label_map = {0: "Human", 1: "AI"}
allow_ext = {".jpg", ".jpeg", ".png", ".bmp", ".webp"}
img_paths = sorted(glob.glob(os.path.join(IMG_DIR, "*.*")))

processed = 0
for p in img_paths:
    if os.path.splitext(p)[1].lower() not in allow_ext:
        continue
    bgr = cv2.imread(p)
    if bgr is None:
        print(f"[Skip] Cannot read: {p}")
        continue
    bgr_resized = cv2.resize(bgr, IMG_SIZE)

    ximg = image.img_to_array(cv2.cvtColor(bgr_resized, cv2.COLOR_BGR2RGB))
    ximg = np.expand_dims(ximg, axis=0)
    ximg = preprocess_input(ximg)  # VGG16 preprocessing

    heatmap, prob_ai = make_gradcam_heatmap(ximg, use_logit=False)
    pred_cls = 1 if prob_ai >= 0.5 else 0

    cam_bgr = overlay_heatmap(heatmap, bgr_resized, alpha=0.45)

    # Convert to RGB for display with matplotlib
    rgb_orig = cv2.cvtColor(bgr_resized, cv2.COLOR_BGR2RGB)
    rgb_cam  = cv2.cvtColor(cam_bgr,     cv2.COLOR_BGR2RGB)

    # Plot and save
    plt.figure(figsize=(6.5, 3.2), dpi=200)
    plt.subplot(1, 2, 1); plt.imshow(rgb_orig); plt.axis("off"); plt.title("Original", fontsize=9)
    plt.subplot(1, 2, 2); plt.imshow(rgb_cam);  plt.axis("off")
    plt.title(f"Grad-CAM — Pred: {label_map[pred_cls]} (AI prob={prob_ai:.2f})", fontsize=9)
    plt.tight_layout()

    # Save
    out_name = os.path.splitext(os.path.basename(p))[0] + "_cam_vgg16.jpg"
    out_path = os.path.join(OUT_DIR, out_name)
    plt.savefig(out_path, bbox_inches="tight")

    # Show inline in notebook
    plt.show()

    processed += 1
    print(f"[Saved and displayed] {out_path}")

print(f"Done. Exported and displayed {processed} VGG16 Grad-CAM images → {OUT_DIR}")


Output hidden; open in https://colab.research.google.com to view.